In [218]:
import numpy as np
from tqdm import trange

In [219]:
def get_node_id(i_row, i_col, i_layer, n_row, n_col, n_layer = None):
    """
    根据行列层编号获取节点编号
    :param i_row: 行编号
    :param i_col: 列编号
    :param i_layer: 层编号
    :param n_row: 行数
    :param n_col: 列数
    :param n_layer: 层数
    :return: 节点编号
    """
    return i_layer*n_row*n_col + i_row*n_col + i_col + 1
def create_msh_from_array(x_array, y_array, z_array, file_path):
    """
    从xyz的坐标数组中创建结构化msh网格
    :param x_array:x的一维数组，shape=(n_col,)
    :param y_array:y的一维数组，shape=(n_row,)
    :param z_array:z的三维数组，shape=(n_row,n_col,n_layer)
    :param file_path:文件路径
    """
    n_row = z_array.shape[0]
    n_col = z_array.shape[1]
    n_layer = z_array.shape[2]
    # 检查输入的数组是否符合要求
    if x_array.shape[0] != n_col:
        raise ValueError(f'x_array.shape[0] != n_col({n_col})')
    if y_array.shape[0] != n_row:
        raise ValueError(f'y_array.shape[0] != n_row({n_row})')
    with open(file_path, 'w') as f:
        f.write('$MeshFormat\n')
        f.write('2.2 0 8\n')
        f.write('$EndMeshFormat\n')
        f.write('$PhysicalNames\n')
        f.write('6\n')
        f.write('2 1 "terrain"\n')
        f.write('2 2 "zmax"\n')
        f.write('2 3 "xmin"\n')
        f.write('2 4 "xmax"\n')
        f.write('2 5 "ymin"\n')
        f.write('2 6 "ymax"\n')
        f.write('$EndPhysicalNames\n')
        f.write('$Nodes\n')
        f.write(f'{n_row*n_col*n_layer}\n')
        # 按照顺序给每个节点编号，存入临时字符串变量
        print('1. node')
        for i_layer in trange(n_layer):
            for i_row in range(n_row):
                for i_col in range(n_col):
                    node_id = get_node_id(i_row, i_col, i_layer, n_row, n_col, n_layer)
                    f.write(f'{node_id} {x_array[i_col]} {y_array[i_row]} {z_array[i_row,i_col,i_layer]}\n')
        # print('nodes_str:\n', nodes_str)
        f.write('$EndNodes\n')
        f.write('$Elements\n')
        # 按照顺序给每个体单元编号，存入临时字符串变量
        print('2. element')
        f.write(f'{(n_row-1)*(n_col-1)*(n_layer-1) + 2*(n_row-1)*(n_col-1) + 2*(n_row-1)*(n_layer-1) + 2*(n_col-1)*(n_layer-1)}\n')
        element_id = 1
        for i_layer in range(n_layer-1):
            for i_row in range(n_row-1):
                for i_col in range(n_col-1):
                    # 体单元的8个节点编号
                    node_id_1 = get_node_id(i_row, i_col, i_layer, n_row, n_col, n_layer)
                    node_id_2 = node_id_1 + 1
                    node_id_3 = node_id_2 + n_col
                    node_id_4 = node_id_1 + n_col
                    node_id_5 = node_id_1 + n_row*n_col
                    node_id_6 = node_id_2 + n_row*n_col
                    node_id_7 = node_id_3 + n_row*n_col
                    node_id_8 = node_id_4 + n_row*n_col
                    f.write(f'{element_id} 5 2 0 1 {node_id_1} {node_id_2} {node_id_3} {node_id_4} {node_id_5} {node_id_6} {node_id_7} {node_id_8}\n')
                    element_id += 1
        # print('elements_str:\n', elements_str)
        # 按照顺序给每个底面单元编号，存入临时字符串变量
        print('3. bottom element')
        for i_row in range(n_row-1):
            for i_col in range(n_col-1):
                # 底面单元的4个节点编号
                node_id_1 = get_node_id(i_row, i_col, 0, n_row, n_col, n_layer)
                node_id_2 = node_id_1 + 1
                node_id_3 = node_id_2 + n_col
                node_id_4 = node_id_1 + n_col
                f.write(f'{element_id} 3 2 1 1 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n') # 第一个1表示底面边界条件组，第二个1表示地面几何组
                element_id += 1
        # print('bottom_elements_str:\n', bottom_elements_str)
        # 按照顺序给每个zmax单元（即i_layer为n_layer-1的面）编号，存入临时字符串变量
        print('4. zmax element')
        for i_row in range(n_row-1):
            for i_col in range(n_col-1):
                # zmax单元的4个节点编号
                node_id_1 = get_node_id(i_row, i_col, n_layer-1, n_row, n_col, n_layer)
                node_id_2 = node_id_1 + 1
                node_id_3 = node_id_2 + n_col
                node_id_4 = node_id_1 + n_col
                f.write(f'{element_id} 3 2 2 2 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n')
                element_id += 1
        # 按照顺序给每个xmin单元（即n_col为0的面）编号，存入临时字符串变量
        print('5. xmin element')
        for i_layer in range(n_layer-1):
            for i_row in range(n_row-1):
                # xmin单元的4个节点编号
                node_id_1 = get_node_id(i_row, 0, i_layer, n_row, n_col, n_layer)
                node_id_2 = node_id_1 + n_row*n_col
                node_id_3 = node_id_2 + n_col
                node_id_4 = node_id_1 + n_col
                f.write(f'{element_id} 3 2 3 3 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n') # 第一个1表示底面边界条件组，第二个1表示地面几何组
                element_id += 1
        # print('xmin_elements_str:\n', xmin_elements_str)
        # 按照顺序给每个xmax单元（即n_col为n_col-1的面）编号，存入临时字符串变量
        print('6. xmax element')
        for i_layer in range(n_layer-1):
            for i_row in range(n_row-1):
                # xmax单元的4个节点编号
                node_id_1 = get_node_id(i_row, n_col-1, i_layer, n_row, n_col, n_layer)
                node_id_2 = node_id_1 + n_row*n_col
                node_id_3 = node_id_2 + n_col
                node_id_4 = node_id_1 + n_col
                f.write(f'{element_id} 3 2 4 4 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n') # 第一个1表示底面边界条件组，第二个1表示地面几何组
                element_id += 1
        # print('xmax_elements_str:\n', xmax_elements_str)
        # 按照顺序给每个ymin单元（即n_row为0的面）编号，存入临时字符串变量
        print('7. ymin element')
        for i_layer in range(n_layer-1):
            for i_col in range(n_col-1):
                # ymin单元的4个节点编号
                node_id_1 = get_node_id(0, i_col, i_layer, n_row, n_col, n_layer)
                node_id_2 = node_id_1 + n_row*n_col
                node_id_3 = node_id_2 + 1
                node_id_4 = node_id_1 + 1
                f.write(f'{element_id} 3 2 5 5 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n') # 第一个1表示底面边界条件组，第二个1表示地面几何组
                element_id += 1
        # print('ymin_elements_str:\n', ymin_elements_str)
        # 按照顺序给每个ymax单元（即n_row为n_row-1的面）编号，存入临时字符串变量
        print('8. ymax element')
        for i_layer in range(n_layer-1):
            for i_col in range(n_col-1):
                # ymax单元的4个节点编号
                node_id_1 = get_node_id(n_row-1, i_col, i_layer, n_row, n_col, n_layer)
                node_id_2 = node_id_1 + n_row*n_col
                node_id_3 = node_id_2 + 1
                node_id_4 = node_id_1 + 1
                f.write(f'{element_id} 3 2 6 6 {node_id_1} {node_id_2} {node_id_3} {node_id_4}\n') # 第一个1表示底面边界条件组，第二个1表示地面几何组
                element_id += 1
        # print('ymax_elements_str:\n', ymax_elements_str)
        f.write('$EndElements\n')

更完整的输入输出（只输入地面的高程和想要的流场高度）

In [220]:
# 输入高程数组和xy坐标数组
dem_array = np.array([[1   , 1.25, 1.1],
                      [1.15, 1.5 , 1.2]])
x_array = np.array([1, 2, 3])
y_array = np.array([0, 1.5])

# 边界层内相关设置
tkns = 0.1 # 地表边界层的最底层厚度
n_layer = 5 # 地表边界层的层数
layer_exponent = 1.2 # 地表边界层的厚度按照指数膨胀
# 边界层外相关设置
z_max = 5 # 最大高程
n_outlayer_need = 20 # 边界层外的层数



dem_array = np.flipud(dem_array) # 翻转数组，使得dem_array[0,0]对应的是左下角的点
# 生成边界层内的z_layers_array（这里实际上只生成了n_layer-1层，最后一层会在后面画边界层外网格时作为第一层画出来）
z_layers_array = np.stack([dem_array+tkns*(1-layer_exponent**i)/(1-layer_exponent) for i in range(n_layer)], axis=2)
# 取z_layer_array的最后一维的最后一个数组成数组z_layer_top_array，即边界层的最上一层
z_layer_top_array = z_layers_array[:,:,-1]


每个元素高度上的网格高度满足等比数列，共n_outlayer项，第一项为边界层顶层厚度，总和为z_max-这个位置处的边界层顶层坐标

In [221]:
z_0 = z_layer_top_array
tkns_1 = tkns * layer_exponent**(n_layer-1) # 边界层最后一层也是边界层外第一层的厚度
tkns_all = z_max - z_0 # 边界层外所有层的厚度
k_array = tkns_all/tkns_1 # 厚度比数组，也是作为方程的参数


计算边界层外每个xy点的膨胀率组成数组

In [222]:
from concurrent.futures import ThreadPoolExecutor
from scipy.optimize import newton
n_outlayer = n_outlayer_need+1
def f(q, k):
    return (q**n_outlayer - 1) / (q - 1) - k

def solve_for_k(k):
    # 设置合适的初始猜测值
    initial_guess = 2.2
    try:
        return newton(f, initial_guess, args=(k,), maxiter=10000)
    except RuntimeError:
        # 处理不收敛的情况
        return np.nan


# 假设 k_values 是您的 NumPy 数组
k_values = k_array.flatten()

# 获取独特的 k 值和它们的索引
unique_ks, indices = np.unique(k_values, return_inverse=True)

# 并行处理独特的 k 值
with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(solve_for_k, unique_ks))

# 将结果映射回原始数组
q_array = np.array(results)[indices].reshape(k_array.shape)
# 如果有nan，提示
if np.any(np.isnan(q_array)):
    print('warning: some q is nan')

基于膨胀率数组计算每个xy点上方的每个网格点的z坐标

In [223]:
# 计算边界层外每一层的z值
q_array_reshaped = q_array.reshape(*q_array.shape, 1)
n_outlayer_list = np.arange(n_outlayer).reshape(1, 1, -1)
tkns_array = tkns_1 * q_array_reshaped**n_outlayer_list
z_0_reshaped = z_0.reshape(*z_0.shape, 1)
z_outlayers_array = z_0_reshaped + np.cumsum(tkns_array, axis=2)

# 将z_outlayers_array堆叠到z_layers_array的最后一维，得到z_array
z_array = np.concatenate([z_layers_array, z_outlayers_array], axis=2)

In [224]:
create_msh_from_array(x_array, y_array, z_array, 'test.msh')

1. node


100%|██████████| 26/26 [00:00<00:00, 26082.73it/s]

2. element
3. bottom element
4. zmax element
5. xmin element
6. xmax element
7. ymin element
8. ymax element


真实地形数据量的测试

In [225]:
# 输入高程数组和xy坐标数组
dem_array = np.load('../mzs_12.5m.npy')
x_array = np.arange(0, dem_array.shape[1]*12.5, 12.5)
y_array = np.arange(0, dem_array.shape[0]*12.5, 12.5)

# 边界层内相关设置
tkns = 0.5 # 地表边界层的最底层厚度
n_layer = 10 # 地表边界层的层数
layer_exponent = 1.2 # 地表边界层的厚度按照指数膨胀
# 边界层外相关设置
z_max = dem_array.max()+1000 # 最大高程
n_outlayer_need = 100 # 边界层外的层数



dem_array = np.flipud(dem_array) # 翻转数组，使得dem_array[0,0]对应的是左下角的点
# 生成边界层内的z_layers_array（这里实际上只生成了n_layer-1层，最后一层会在后面画边界层外网格时作为第一层画出来）
z_layers_array = np.stack([dem_array+tkns*(1-layer_exponent**i)/(1-layer_exponent) for i in range(n_layer)], axis=2)
# 取z_layer_array的最后一维的最后一个数组成数组z_layer_top_array，即边界层的最上一层
z_layer_top_array = z_layers_array[:,:,-1]





z_0 = z_layer_top_array
tkns_1 = tkns * layer_exponent**(n_layer-1) # 边界层最后一层也是边界层外第一层的厚度
tkns_all = z_max - z_0 # 边界层外所有层的厚度
k_array = tkns_all/tkns_1 # 厚度比数组，也是作为方程的参数





from concurrent.futures import ThreadPoolExecutor
from scipy.optimize import newton
n_outlayer = n_outlayer_need+1
def f(q, k):
    return (q**n_outlayer - 1) / (q - 1) - k

def solve_for_k(k):
    # 设置合适的初始猜测值
    initial_guess = 2.2
    try:
        return newton(f, initial_guess, args=(k,), maxiter=10000)
    except RuntimeError:
        # 处理不收敛的情况
        return np.nan


# 假设 k_values 是您的 NumPy 数组
k_values = k_array.flatten()

# 获取独特的 k 值和它们的索引
unique_ks, indices = np.unique(k_values, return_inverse=True)

# 并行处理独特的 k 值
with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(solve_for_k, unique_ks))

# 将结果映射回原始数组
q_array = np.array(results)[indices].reshape(k_array.shape)
# 如果有nan，提示
if np.any(np.isnan(q_array)):
    print('warning: some q is nan')




# 计算边界层外每一层的z值
q_array_reshaped = q_array.reshape(*q_array.shape, 1)
n_outlayer_list = np.arange(n_outlayer).reshape(1, 1, -1)
tkns_array = tkns_1 * q_array_reshaped**n_outlayer_list
z_0_reshaped = z_0.reshape(*z_0.shape, 1)
z_outlayers_array = z_0_reshaped + np.cumsum(tkns_array, axis=2)

# 将z_outlayers_array堆叠到z_layers_array的最后一维，得到z_array
z_array = np.concatenate([z_layers_array, z_outlayers_array], axis=2)




create_msh_from_array(x_array, y_array, z_array, 'terrain.msh')

1. node


100%|██████████| 111/111 [00:01<00:00, 74.86it/s]


2. element
3. bottom element
4. zmax element
5. xmin element
6. xmax element
7. ymin element
8. ymax element
